In [ ]:
# Import standard modules.
from importlib import import_module
import os
import sys

# Import supplemental modules.
import matplotlib.pyplot as plt
import numpy as np

# Import project modules.
import pinn.standard_plots as psp

In [ ]:
# Specify the run ID (aka problem name).
runid = "loop2d_nPBxByBz"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Extract indices for independent and dependent variables.
it = p.independent_variable_index["t"]
ix = p.independent_variable_index["x"]
iy = p.independent_variable_index["y"]
i_n = p.dependent_variable_index["n"]  # Underscore needed to avoid name collision with "in".
iP = p.dependent_variable_index["P"]
iBx = p.dependent_variable_index["Bx"]
iBy = p.dependent_variable_index["By"]
iBz = p.dependent_variable_index["Bz"]

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Define analytical solutions.

# Constants
Q = 60.0   # Flow angle in degrees clockwise from +y axis
u0 = 1.0   # Initial flow speed
R0 = 0.3   # Radius of current tube
A = 1e-3   # Maximum magnitude of vector potential  in z-direction
           # A_vec = A*MAX(R0 - r, 0) z_hat

# Compute the constant velocity components.
u0x = u0*np.sin(np.radians(Q))
u0y = u0*np.cos(np.radians(Q))

# Constant values for other variables.
n0 = 1.0
P0 = 1.0
B0z = 0.0


def n_analytical(t, x, y):
    n = np.full(t.shape, n0)
    return n

def P_analytical(t, x, y):
    P = np.full(t.shape, P0)
    return P

def Bx_analytical(t, x, y):
    xp = x - u0x*t
    yp = y - u0y*t
    r = np.sqrt(xp**2 + yp**2)
    w = np.where(r < R0)
    Bx = np.zeros(t.shape[0])
    Bx[w] = -A*yp[w]/r[w]
    return Bx

def By_analytical(t, x, y):
    xp = x - u0x*t
    yp = y - u0y*t
    r = np.sqrt(xp**2 + yp**2)
    w = np.where(r < R0)
    By = np.zeros(t.shape[0])
    By[w] = A*xp[w]/r[w]
    return By

def Bz_analytical(t, x, y):
    Bz = np.full(t.shape, B0z)
    return Bz

def dBx_dx_analytical(t, x, y):
    xp = x - u0x*t
    yp = y - u0y*t
    r = np.sqrt(xp**2 + yp**2)
    w = np.where(r < R0)
    dBx_dx = np.zeros(t.shape[0])
    dBx_dx[w] = A*xp[w]*yp[w]/r[w]**3
    return dBx_dx

def dBy_dy_analytical(t, x, y):
    xp = x - u0x*t
    yp = y - u0y*t
    r = np.sqrt(xp**2 + yp**2)
    w = np.where(r < R0)
    dBy_dy = np.zeros(t.shape[0])
    dBy_dy[w] = -A*xp[w]*yp[w]/r[w]**3
    return dBy_dy

In [ ]:
# Load all data.

# Load the training point coordinates.
txy_train = np.loadtxt(os.path.join(runid, "X_train.dat"))
t_train = txy_train[:, it]
x_train = txy_train[:, ix]
y_train = txy_train[:, iy]

# Load the data locations and values (includes initial conditions).
txy_data = np.loadtxt(os.path.join(runid, "XY_data.dat"))

# Extract the initial conditions.
ic = txy_data[:, p.n_dim:]

# Load the model-predicted values.
ψ = []
delψ = []
for var_name in p.dependent_variable_names:
    ψ.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delψ.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))

# Load the loss function histories.
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_model_res = np.loadtxt(os.path.join(runid, "losses_model_res.dat"))
losses_model_data = np.loadtxt(os.path.join(runid, "losses_model_data.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))
losses_res = np.loadtxt(os.path.join(runid, "losses_res.dat"))
losses_data = np.loadtxt(os.path.join(runid, "losses_data.dat"))

In [ ]:
# Compute the limits of the training domain.
t_min = t_train[0]
t_max = t_train[-1]
x_min = x_train[0]
x_max = x_train[-1]
y_min = y_train[0]
y_max = y_train[-1]

# Extract the unique training point values (a grid is assumed).
t_train_vals = np.unique(t_train)
x_train_vals = np.unique(x_train)
y_train_vals = np.unique(y_train)
n_t_train_vals = len(t_train_vals)
n_x_train_vals = len(x_train_vals)
n_y_train_vals = len(y_train_vals)

In [ ]:
# Plotting options

# Specify the size (inches) for individual subplots.
SUBPLOT_WIDTH = 5.0
SUBPLOT_HEIGHT = 5.0

# Compute the figure size for the total loss plot.
total_loss_figsize = (SUBPLOT_WIDTH*2, SUBPLOT_HEIGHT)

# Compute the coordinate plot tick locations and labels.
XY_N_X_TICKS = 5
XY_x_tick_pos = np.linspace(x_min, x_max, XY_N_X_TICKS)
XY_x_tick_labels = ["%.1f" % x for x in XY_x_tick_pos]
XY_N_Y_TICKS = 5
XY_y_tick_pos = np.linspace(y_min, y_max, XY_N_Y_TICKS)
XY_y_tick_labels = ["%.1f" % y for y in XY_y_tick_pos]

# Compute the heat map tick locations and labels.
HEATMAP_N_X_TICKS = 5
heatmap_x_tick_pos = np.linspace(0, n_x_train_vals - 1, HEATMAP_N_X_TICKS)
heatmap_x_tick_labels = ["%.1f" % (x_min + x/(n_x_train_vals - 1)*(x_max - x_min)) for x in heatmap_x_tick_pos]
HEATMAP_N_Y_TICKS = 5
heatmap_y_tick_pos = np.linspace(0, n_y_train_vals - 1, HEATMAP_N_Y_TICKS)
heatmap_y_tick_labels = ["%.1f" % (y_min + y/(n_y_train_vals - 1)*(y_max - y_min)) for y in heatmap_y_tick_pos]
heatmap_y_tick_labels = list(reversed(heatmap_y_tick_labels))

In [ ]:
# Plot the loss history for each model.
fig = psp.plot_model_loss_functions(
    losses_model_res, losses_model_data, losses_model,
    p.dependent_variable_labels
)

In [ ]:
# Plot the total loss function history.
plt.figure(figsize=total_loss_figsize)
psp.plot_loss_functions(
    [losses_res, losses_data, losses],
    ["$L_{res}$", "$L_{data}$", "$L$"],
    title="Total loss function history for %s" % runid
)

In [ ]:
# Extract the coordinates for the training points at the initial time.
n_start = n_x_train_vals*n_y_train_vals
t = txy_data[:, it]
x = txy_data[:, ix]
y = txy_data[:, iy]

In [ ]:
# Plot the actual and predicted initial magnetic field vectors.
# This code assumes all of the data coordinates are the first n_start training coordinates.
B0x_act = Bx_analytical(t, x, y)
B0y_act = By_analytical(t, x, y)
B0x_pred = ψ[iBx][:n_start]
B0y_pred = ψ[iBy][:n_start]

# Create the figure.
fig = psp.plot_actual_predicted_B(
    x, y, B0x_act, B0y_act, B0x_pred, B0y_pred,
    title="Initial magnetic field",
    x_tick_pos=XY_x_tick_pos, x_tick_labels=XY_x_tick_labels,
    y_tick_pos=XY_y_tick_pos, y_tick_labels=XY_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in initial magnetic field magnitudes.
B0_act = np.sqrt(B0x_act**2 + B0y_act**2)
B0_pred = np.sqrt(B0x_pred**2 + B0y_pred**2)
B0_err = B0_pred - B0_act

# To get the proper orientation, reshape, transpose, flip.
B0_act_plot = np.flip(B0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0_pred_plot = np.flip(B0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0_err_plot = np.flip(B0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the plot.
fig = psp.plot_log_actual_predicted_error(
    x, y, B0_act_plot, B0_pred_plot, B0_err_plot,
    vmin=1e-6, vmax=1, err_vmin=-1e-3, err_vmax=1e-3,
    title="Initial magnetic field magnitude",
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in initial magnetic field divergence.
dB0x_dx_act = dBx_dx_analytical(t, x, y)
dB0y_dy_act = dBy_dy_analytical(t, x, y)
divB0_act = dB0x_dx_act + dB0y_dy_act
dB0x_dx_pred = delψ[iBx][:n_start, ix]
dB0y_dy_pred = delψ[iBy][:n_start, iy]
divB0_pred = dB0x_dx_pred + dB0y_dy_pred
divB0_err = divB0_pred - divB0_act

# To get the proper orientation, reshape, transpose, flip.
divB0_act_plot = np.flip(divB0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
divB0_pred_plot = np.flip(divB0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
divB0_err_plot = np.flip(divB0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, divB0_act_plot, divB0_pred_plot, divB0_err_plot,
    vmin=-1e-3, vmax=1e-3, err_vmin=-1e-3, err_vmax=1e-3,
    title="Initial magnetic divergence",
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in initial number density.
n0_act = n_analytical(t, x, y)
n0_pred = ψ[i_n][:n_start]
n0_err = n0_pred - n0_act

# To get the proper orientation, reshape, transpose, flip.
n0_act_plot = np.flip(n0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
n0_pred_plot = np.flip(n0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
n0_err_plot = np.flip(n0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, n0_act_plot, n0_pred_plot, n0_err_plot,
    title="Initial %s" % p.dependent_variable_labels[i_n],
    vmin=n0 - 0.1, vmax=n0 + 0.1, err_vmin=-0.1, err_vmax=0.1,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in initial pressure.
P0_act = P_analytical(t, x, y)
P0_pred = ψ[iP][:n_start]
P0_err = P0_pred - P0_act

# To get the proper orientation, reshape, transpose, flip.
P0_act_plot = np.flip(P0_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
P0_pred_plot = np.flip(P0_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
P0_err_plot = np.flip(P0_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, P0_act_plot, P0_pred_plot, P0_err_plot,
    title="Initial %s" % p.dependent_variable_labels[iP],
    vmin=P0 - 0.1, vmax=P0 + 0.1, err_vmin=-0.1, err_vmax=0.1,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in initial Bz.
B0z_act = Bz_analytical(t, x, y)
B0z_pred = ψ[iBz][:n_start]
B0z_err = B0z_pred - B0z_act

# To get the proper orientation, reshape, transpose, flip.
B0z_act_plot = np.flip(B0z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0z_pred_plot = np.flip(B0z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B0z_err_plot = np.flip(B0z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, B0z_act_plot, B0z_pred_plot, B0z_err_plot,
    title="Initial %s" % p.dependent_variable_labels[iBz],
    vmin=-1e-6, vmax=1e-6, err_vmin=-1e-6, err_vmax=1e-6,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Extract the coordinates of the training points at the final time.
n_end = n_x_train_vals*n_y_train_vals
t = t_train[-n_end:]
x = x_train[-n_end:]
y = y_train[-n_end:]

In [ ]:
# Plot the actual and predicted final magnetic field vectors.
B1x_act = Bx_analytical(t, x, y)
B1y_act = By_analytical(t, x, y)
B1x_pred = ψ[iBx][-n_end:]
B1y_pred = ψ[iBy][-n_end:]

# Create the figure.
fig = psp.plot_actual_predicted_B(
    x, y, B1x_act, B1y_act, B1x_pred, B1y_pred,
    title="Final magnetic field",
    x_tick_pos=XY_x_tick_pos, x_tick_labels=XY_x_tick_labels,
    y_tick_pos=XY_y_tick_pos, y_tick_labels=XY_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in final magnetic field magnitude.
B1_act = np.sqrt(B1x_act**2 + B1y_act**2)
B1_pred = np.sqrt(B1x_pred**2 + B1y_pred**2)
B1_err = B1_pred - B1_act

# To get the proper orientation, reshape, transpose, flip.
B1_act_plot = np.flip(B1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B1_pred_plot = np.flip(B1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B1_err_plot = np.flip(B1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the plot.
fig = psp.plot_log_actual_predicted_error(
    x, y, B1_act_plot, B1_pred_plot, B1_err_plot,
    vmin=1e-6, vmax=1, err_vmin=-1e-3, err_vmax=1e-3,
    title="Final magnetic field magnitude",
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in final magnetic field divergence.
dB1x_dx_act = dBx_dx_analytical(t, x, y)
dB1y_dy_act = dBy_dy_analytical(t, x, y)
divB1_act = dB1x_dx_act + dB1y_dy_act
dB1x_dx_pred = delψ[iBx][-n_end:, ix]
dB1y_dy_pred = delψ[iBy][-n_end:, iy]
divB1_pred = dB1x_dx_pred + dB1y_dy_pred
divB1_err = divB1_pred - divB1_act

# To get the proper orientation, reshape, transpose, flip.
divB1_act_plot = np.flip(divB1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
divB1_pred_plot = np.flip(divB1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
divB1_err_plot = np.flip(divB1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, divB1_act_plot, divB1_pred_plot, divB1_err_plot,
    vmin=-1e-3, vmax=1e-3, err_vmin=-1e-3, err_vmax=1e-3,
    title="Final magnetic divergence",
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in final number density.
n1_act = n_analytical(t, x, y)
n1_pred = ψ[i_n][-n_end:]
n1_err = n1_pred - n1_act

# To get the proper orientation, reshape, transpose, flip.
n1_act_plot = np.flip(n1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
n1_pred_plot = np.flip(n1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
n1_err_plot = np.flip(n1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, n1_act_plot, n1_pred_plot, n1_err_plot,
    title="Final %s" % p.dependent_variable_labels[i_n],
    vmin=n0 - 0.1, vmax=n0 + 0.1, err_vmin=-0.1, err_vmax=0.1,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in final pressure.
P1_act = P_analytical(t, x, y)
P1_pred = ψ[iP][-n_end:]
P1_err = P1_pred - P1_act

# To get the proper orientation, reshape, transpose, flip.
P1_act_plot = np.flip(P1_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
P1_pred_plot = np.flip(P1_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
P1_err_plot = np.flip(P1_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, P1_act_plot, P1_pred_plot, P1_err_plot,
    title="Final %s" % p.dependent_variable_labels[iP],
    vmin=P0 - 0.1, vmax=P0 + 0.1, err_vmin=-0.1, err_vmax=0.1,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)

In [ ]:
# Plot the actual, predicted, and absolute error in final Bz.
B1z_act = Bz_analytical(t, x, y)
B1z_pred = ψ[iBz][-n_end:]
B1z_err = B1z_pred - B1z_act

# To get the proper orientation, reshape, transpose, flip.
B1z_act_plot = np.flip(B1z_act.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B1z_pred_plot = np.flip(B1z_pred.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)
B1z_err_plot = np.flip(B1z_err.reshape(n_x_train_vals, n_y_train_vals).T, axis=0)

# Create the figure.
fig = psp.plot_actual_predicted_error(
    x, y, B1z_act_plot, B1z_pred_plot, B1z_err_plot,
    title="Initial %s" % p.dependent_variable_labels[iBz],
    vmin=-1e-6, vmax=1e-6, err_vmin=-1e-6, err_vmax=1e-6,
    x_tick_pos=heatmap_x_tick_pos, x_tick_labels=heatmap_x_tick_labels,
    y_tick_pos=heatmap_y_tick_pos, y_tick_labels=heatmap_y_tick_labels,
)